# With the money I have, which is the best district to invest on? 

You have 10 points to distribute across three variables. Please divide the points across these three variables according to what you give more importance when doing an investment.

- **PER** number of years to get the return of investment.
- **RB** Anual rental return.
- **revaluation** Increase in property value. The apartment price after the PER years therefore the original investment is recovered.


> Columns information:

- advertised_rent: the average advertised time of the apartments for renting in this group is 1.244643 months
- advertised_sell: the average advertised time of the apartments for sale in this group is 2.062019 months
- size_rent: number of apartments for renting with these characteristics
- size_sell: number of apartments for sale with these characteristics
- €/meter_rent: Average renting price per square meter 
- €/meter_sell: Average selling price per square meter 
- apartments: total number of apartments, in alcala-de-henares. This value can be used to calculate the percentage of apartments for sale and for rent from each district
- RB: annual rental yield compared to the purchase price. Formula: RB = ((€/meter_rent*12) / €/meter_sell) * 100
- PER (Price-to-Earnings): number of years to recover the sale price with the rent. Formula PER = €/meter_sell / (€/meter_rent*12)
- change_2014_2024_sell: average change rate in sale price from 2014 to 2024
- change_2014_2024_rent: average change rate in renting price from 2014 to 2024
- change_2021_2024_sell: average change rate in sale price from 2021 to 2024
- change_2021_2024_rent: average change rate in renting price from 2021 to 2024
- sell: sale prince: mean_meters * €/meter_sell
- rent: renting prince: mean_meters * €/meter_rent
- revaluation: how much does the apartment cost after the PER years have passed and applying the change_2021_2024_sell of price evolution.
- PER_adjusted: PER adjusted taking into account that the rent price will increase based on change_2021_2024_rent during the PER years.


## Code

In [37]:
import pandas as pd
import geopandas as gpd
from pysal.lib import weights
from sklearn.preprocessing import MinMaxScaler


pd.set_option('display.float_format', '{:.3f}'.format)


def calculate_investment_score(data, input_money, weight_revaluation, weight_rent_anual, weight_per):
    try:
        data['loan'] = data['sell'].apply(lambda x: x - float(input_money)) # keep just the ones with 0 or positive loan
        filtered = data[data['loan'] > 0].copy()
        scaler = MinMaxScaler()
        filtered['normalized_loan'] = scaler.fit_transform(1 / filtered[['loan']])
        filtered['score'] = (
                            float(weight_rent_anual)/10 * filtered['norm_RB']
                             + float(weight_revaluation)/10 * filtered['norm_revaluation']
                             + float(weight_per)/10 * filtered['norm_PER']
                            )
        cols = ['distrito', 'size_rent', 'size_sell', 'mean_meters', 'rent', 'sell', 'RB', 'PER', 'score', 'revaluation', 'loan', 'advertised_rent_time', 'advertised_sell_time']
        filtered = filtered[cols]
        filtered = filtered.sort_values(by=['loan'])
    except Exception as e:
        print('Please enter valid numbers', e)
    return filtered


data = pd.read_csv('../data/info_per_district_and_meters.csv')

maps = gpd.read_file('../data/maps.geojson')
df = gpd.GeoDataFrame(pd.merge(result, maps, on='distrito'))
df = df.drop_duplicates(subset=['distrito'], keep='first')

# Calculate weights
w = weights.Queen.from_dataframe(df, idVariable='distrito')
w.transform = 'R'

## Input

In [49]:
input_money = input('Enter the money you want to invest:')

weight_revaluation = input('How many points do you give to revaluation?')
weight_rent_anual = input('How many points do you give to RB (Gross annual yield)?')
weight_per = input('How many points do you give to PER?')

questions_and_answers = {
    'Enter the money you want to invest:': input_money,
    'How many points do you give to revaluation?': weight_revaluation,
    'How many points do you give to RB (Gross annual yield)?': weight_rent_anual,
    'How many points do you give to PER?': weight_per,
}

print("\nYour inputs:")
for question, answer in questions_and_answers.items():
    print(f"{question} {answer}")

result = calculate_investment_score(data, input_money, weight_revaluation, weight_rent_anual, weight_per)
result.head(20)

Enter the money you want to invest: 50000
How many points do you give to revaluation? 2
How many points do you give to RB (Anual rental return)? 6
How many points do you give to PER? 2
Number of columns you want to plot 50



Your inputs:
Enter the money you want to invest: 50000
How many points do you give to revaluation? 2
How many points do you give to RB (Gross annual yield)? 6
How many points do you give to PER? 2


,distrito,size_rent,size_sell,mean_meters,rent,sell,RB,PER,score,revaluation,loan,advertised_rent_time,advertised_sell_time
83,el-escorial,1.000,1.000,30.000,428.571,68571.429,7.500,13.333,0.446,1107483.550,18571.429,1.000,2.000
153,parla,0.000,1.000,30.000,0.000,72729.730,0.000,0.000,0.000,72729.730,22729.730,0.000,2.000
101,getafe,1.000,2.000,30.000,528.000,78025.210,8.120,12.315,0.482,687094.184,28025.210,1.533,1.510
0,alcala-de-henares,0.000,2.000,30.000,0.000,94928.571,0.000,0.000,0.000,94928.571,44928.571,0.000,1.217
168,puente-de-vallecas,15.000,59.000,30.000,992.682,98549.747,12.087,8.273,0.718,648693.863,48549.747,1.295,2.012
45,carabanchel,23.000,21.000,30.000,1141.744,102666.990,13.345,7.493,0.793,589407.856,52666.990,1.565,2.276
252,villaviciosa-de-odon,9.000,3.000,50.000,1167.130,104000.000,13.467,7.426,0.800,286837.525,54000.000,1.532,4.000
70,collado-villalba,2.000,2.000,30.000,628.378,113850.357,6.623,15.098,0.393,1761333.462,63850.357,1.417,1.033
10,alcorcon,0.000,2.000,30.000,0.000,115384.615,0.000,0.000,0.000,115384.615,65384.615,0.000,1.567
246,villaverde,8.000,2.000,30.000,825.626,120357.143,8.232,12.148,0.489,1221041.621,70357.143,1.443,1.500


# Create map with a color map of the score

In [52]:
# Calculate spatial lag for score
df.loc[:, 'score_std'] = (df['score'] - df['score'].mean()) / df['score'].std()
df['w_score_std'] = weights.lag_spatial(w, df['score_std'])
df.explore(column='w_score_std', cmap='viridis')